In [89]:
import numpy as np

class Linear:
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        self.weights = np.random.randn(in_features, out_features) * 0.01
        self.biases = np.zeros((1, out_features))
        self.input = None
        self.grad_weights = None
        self.grad_biases = None

    def forward(self, x):
        self.input = x
        return np.dot(x, self.weights) + self.biases

    def backward(self, d_out):
        self.grad_weights = np.dot(self.input.T, d_out)
        self.grad_biases = np.sum(d_out, axis=0, keepdims=True)
        d_input = np.dot(d_out, self.weights.T)
        return d_input


In [90]:
class ReLU:
    def forward(self, x):
        self.input = x
        return np.maximum(0, x)

    def backward(self, d_out):
        d_input = d_out.copy()
        d_input[self.input <= 0] = 0
        return d_input


In [91]:
class Sigmoid:
    def forward(self, x):
        self.output = 1 / (1 + np.exp(-x))
        return self.output

    def backward(self, d_out):
        return d_out * (self.output * (1 - self.output))


In [92]:
class Tanh:
    def forward(self, x):
        self.output = np.tanh(x)
        return self.output

    def backward(self, d_out):
        return d_out * (1 - self.output ** 2)


In [93]:
class Softmax:
    def forward(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        self.output = exps / np.sum(exps, axis=1, keepdims=True)
        return self.output

    def backward(self, d_out):
        # Assuming d_out is already the gradient of loss w.r.t. softmax output
        return d_out


In [94]:
class CrossEntropyLoss:
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        self.n = y_true.shape[0]
        loss = -np.sum(y_true * np.log(y_pred + 1e-10)) / self.n
        return loss

    def backward(self):
        return (self.y_pred - self.y_true) / self.y_true.shape[0]


In [95]:
class MSELoss:
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return np.mean((y_pred - y_true) ** 2)

    def backward(self):
        return 2 * (self.y_pred - self.y_true) / self.y_true.size


In [96]:
class SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate

    def step(self, layer):
        if hasattr(layer, 'grad_weights'):
            layer.weights -= self.learning_rate * layer.grad_weights
            layer.biases -= self.learning_rate * layer.grad_biases


In [97]:
class Model:
    def __init__(self):
        self.layers = []
        self.loss_fn = None
        self.optimizer = None

    def add_layer(self, layer):
        self.layers.append(layer)

    def compile(self, loss_fn, optimizer):
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, loss_grad):
        for layer in reversed(self.layers):
            loss_grad = layer.backward(loss_grad)

    def train(self, x_train, y_train, epochs, batch_size):
        for epoch in range(epochs):
            # Shuffle training data
            indices = np.arange(x_train.shape[0])
            np.random.shuffle(indices)
            x_train, y_train = x_train[indices], y_train[indices]
            
            # Mini-batch training
            for start in range(0, x_train.shape[0], batch_size):
                end = min(start + batch_size, x_train.shape[0])
                x_batch, y_batch = x_train[start:end], y_train[start:end]
                
                # Forward pass
                predictions = self.forward(x_batch)
                
                # Compute loss
                loss = self.loss_fn.forward(predictions, y_batch)
                
                # Backward pass
                loss_grad = self.loss_fn.backward()
                self.backward(loss_grad)
                
                # Update parameters
                for layer in self.layers:
                    self.optimizer.step(layer)
                
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss}')
            self.display_weights()

    def display_weights(self):
        for i, layer in enumerate(self.layers):
            if hasattr(layer, 'weights'):
                print(f'Layer {i + 1} weights:\n{layer.weights}')
            if hasattr(layer, 'biases'):
                print(f'Layer {i + 1} biases:\n{layer.biases}')

    def evaluate(self, x_test, y_test):
        predictions = self.forward(x_test)
        loss = self.loss_fn.forward(predictions, y_test)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
        return loss, accuracy

    def save(self, filename):
        # Implement model saving
        pass

    def load(self, filename):
        # Implement model loading
        pass


In [98]:
import pandas as pd
from tensorflow.keras.utils import to_categorical

# Load dataset
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

# Separate features and labels
x = train_df.drop(columns='label').values
y = train_df['label'].values

# Normalize pixel values
x = x / 255.0

# Reshape data to (num_samples, 28, 28)
x = x.reshape(-1, 28, 28)

# One-hot encode the labels
y = to_categorical(y, 10)

# Flatten the data
x = x.reshape(x.shape[0], -1)

# Split data into training and validation sets

np.random.seed(42)  # For reproducibility
indices = np.arange(x.shape[0])
np.random.shuffle(indices)
split_index = int(0.8 * len(indices))

train_indices = indices[:split_index]
val_indices = indices[split_index:]

x_train, x_val = x[train_indices], x[val_indices]
y_train, y_val = y[train_indices], y[val_indices]


In [99]:
# Initialize model and layers
model = Model()
model.add_layer(Linear(784, 128))
model.add_layer(ReLU())
model.add_layer(Linear(128, 10))
model.add_layer(Softmax())

# Compile model with loss and optimizer
loss_fn = CrossEntropyLoss()
optimizer = SGD(learning_rate=0.5)
model.compile(loss_fn, optimizer)

# Train the model
model.train(x_train, y_train, epochs=25, batch_size=64)

# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(x_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


Epoch 1/25, Loss: 0.21772892976406097
Layer 1 weights:
[[ 5.93538986e-03  2.60148577e-03 -2.35444458e-03 ...  3.78851229e-03
   1.47141819e-03  4.29591303e-03]
 [-4.41319004e-03  9.29479062e-03  3.61001946e-03 ... -9.82393811e-03
   8.56413506e-03 -1.82450003e-03]
 [ 2.01436544e-03  1.63776871e-03  5.47264251e-04 ...  1.14379761e-03
  -7.87677296e-03 -2.92317146e-03]
 ...
 [-8.91762530e-05 -1.87301902e-03 -2.37005061e-03 ... -2.09615579e-03
   6.75959957e-03  5.09208878e-03]
 [-3.74598464e-03  6.73663176e-03  2.72120217e-02 ...  2.70637627e-03
   3.14580157e-03  7.72825713e-04]
 [-1.67471550e-02 -1.94004075e-03 -1.75420087e-03 ...  8.10525999e-03
  -1.54631799e-02  1.05056601e-02]]
Layer 1 biases:
[[-0.09167512  0.055377    0.03285778 -0.00265917 -0.01227313  0.01082208
   0.05030709 -0.00405591  0.01364748 -0.08419697 -0.07049043  0.10077101
   0.08854419  0.03190927  0.15974093 -0.18691112 -0.04307237 -0.12581117
   0.02811582 -0.045116   -0.09178836  0.04227565 -0.00206268 -0.045789